In [1]:
# NLP
# NLP has the goal of deriving information out of natural language (sequences of text or speech)
# another common term is sequence to sequence seq2seq


In [2]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2025-05-19 17:41:44--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2025-05-19 17:41:44 (72.5 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [4]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [5]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2025-05-19 17:41:54--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.111.207, 142.251.16.207, 142.251.163.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.111.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.008s  

2025-05-19 17:41:54 (74.6 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [6]:
# Turn .csv files into pandas DataFrame's
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) # shuffle with random_state=42 for reproducibility
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
# The test data doesn't have a target (that's what we'd try to predict)
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
# How many examples of each class?
train_df.target.value_counts()

,count
target,
0,4342
1,3271


In [10]:
# How many samples total?
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total test samples: 3263
Total samples: 10876


In [11]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text:
Watch This Airport Get Swallowed Up By A Sandstorm In Under A Minute http://t.co/1tr2KvXCTW

---

Target: 0 (not real disaster)
Text:
#LukeBox something about first responders/ military they are our true Hero's!! Besides your music

---

Target: 0 (not real disaster)
Text:
BODY BAGS! https://t.co/0McXc68GZD

---

Target: 1 (real disaster)
Text:
FedEx not willing to transport research specimens of potential bioterror pathogens in wake of anthrax lab mishaps  http://t.co/cM8UnI1mRG

---

Target: 1 (real disaster)
Text:
Red Cross re-opens shelter at Bickleton School after 25 homes evacuated in &amp; around Roosevelt WA due to wildfire. #LiveOnK2

---



In [12]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42) # random state for reproducibility

In [13]:

# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [14]:
# View the first 10 training sentences and their labels
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

In [15]:
# Converting text to numbers: tokenization | embedding

In [16]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [17]:
text_vectorizer = TextVectorization(max_tokens=None, # how many words in vocabulary add <OOV> if > max_t
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None, #how long do you want your sequences to be
                                    )
                                   # pad_to_max_tokens=True)



In [18]:
# Find the avg number of tokens (words) in the training tweets (divide sum of lengths of each tweet by length of dataset)

In [19]:
sum([len(i.split()) for i in train_sentences]) / len(train_sentences)

14.901036345059115

In [20]:
max_vocab_length=10000 # max number of words to have in our vocab
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [21]:
text_vectorizer.adapt(train_sentences)

In [22]:
sample_senetence = "There's a flood in my street!"
text_vectorizer([sample_senetence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [23]:
random_sentence = random.choice(train_sentences)
print(f"Original: \n {random_sentence} \n\nVectorized: \n {text_vectorizer([random_sentence])}")


Original: 
 11-Year-Old Boy Charged With Manslaughter of Toddler: Report: An 11-year-old boy has been charged with manslaughter over the fatal sh... 

Vectorized: 
 [[752 291 333  14 702   6 803 329  39 752 291  41  59 333  14]]


In [24]:
# get the unique words in the vocab

In [25]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 most common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', np.str_('the'), np.str_('a'), np.str_('in')]
Bottom 5 most common words: [np.str_('pages'), np.str_('paeds'), np.str_('pads'), np.str_('padres'), np.str_('paddytomlinson1')]


In [26]:
# Creating an Embedding using Embedding layer

In [27]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length=max_length)

embedding

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


<Embedding name=embedding, built=False>

In [28]:
random_sentence  = random.choice(train_sentences)
print(f"Original sentence: \n {random_sentence} \n\nEmbedded version: ")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original sentence: 
 CLEARED:incident with injury:I-495  inner loop Exit 31 - MD 97/Georgia Ave Silver Spring 

Embedded version: 


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00947792, -0.01999745, -0.01636275, ..., -0.03155158,
         -0.01704482,  0.01536702],
        [-0.01878076, -0.03411851, -0.00319248, ...,  0.00867767,
          0.04690978,  0.00089215],
        [ 0.02738469,  0.04539244,  0.0078328 , ..., -0.01271886,
          0.04642569, -0.04604045],
        ...,
        [-0.01835526,  0.03917566, -0.00099736, ..., -0.02833693,
          0.01688695,  0.01609775],
        [-0.01835526,  0.03917566, -0.00099736, ..., -0.02833693,
          0.01688695,  0.01609775],
        [-0.01835526,  0.03917566, -0.00099736, ..., -0.02833693,
          0.01688695,  0.01609775]]], dtype=float32)>

In [29]:
### model 0: baseline model sci-kit (Naive Bayes TF-IDF)

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB()), # model the text. clf = classifier
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [31]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

our baseline model achieves an accuracy of: 79.27%


In [32]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [33]:
train_labels

array([0, 0, 1, ..., 1, 1, 0])

In [34]:
## EVALUATION FUNCTION FOR MODEL EXPERIMENTS

In [35]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def caluclate_results(y_true, y_pred):
  """
  Calcualates model accuracy, precision, recall and f1 score of a binary classification model
  """
  model_accurancy = accuracy_score(y_true, y_pred) * 100
  # precision, recall and f1-score using *weighted* average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accurancy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}

  return model_results


In [36]:
baseline_results = caluclate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [37]:
from helper_functions import create_tensorboard_callback

SAVE_DIR = "model_logs"

from tensorflow.keras import layers

inputs = layers.Input(shape=(), dtype=tf.string) # inputs  are 1-dimensional strings
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None)                 │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [38]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [39]:
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20250519-174159
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 26ms/step - accuracy: 0.6365 - loss: 0.6516 - val_accuracy: 0.7572 - val_loss: 0.5364
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.8092 - loss: 0.4608 - val_accuracy: 0.7835 - val_loss: 0.4765
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.8632 - loss: 0.3520 - val_accuracy: 0.7887 - val_loss: 0.4611
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.8961 - loss: 0.2831 - val_accuracy: 0.7835 - val_loss: 0.4743
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.9058 - loss: 0.2422 - val_accuracy: 0.7848 - val_loss: 0.4780


In [40]:
model_1.evaluate(val_sentences, val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7810 - loss: 0.5114


[0.4779607355594635, 0.7847769260406494]

In [41]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


(762, 1)

In [42]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [43]:
model_1_results = caluclate_results(y_true=val_labels, y_pred=model_1_preds)
model_1_results

{'accuracy': 78.4776902887139,
 'precision': 0.7895833902086796,
 'recall': 0.7847769028871391,
 'f1': 0.7817499108999565}

In [44]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

In [45]:
words_in_vocab

['',
 '[UNK]',
 np.str_('the'),
 np.str_('a'),
 np.str_('in'),
 np.str_('to'),
 np.str_('of'),
 np.str_('and'),
 np.str_('i'),
 np.str_('is'),
 np.str_('for'),
 np.str_('on'),
 np.str_('you'),
 np.str_('my'),
 np.str_('with'),
 np.str_('it'),
 np.str_('that'),
 np.str_('at'),
 np.str_('by'),
 np.str_('this'),
 np.str_('from'),
 np.str_('be'),
 np.str_('are'),
 np.str_('was'),
 np.str_('have'),
 np.str_('like'),
 np.str_('as'),
 np.str_('up'),
 np.str_('so'),
 np.str_('just'),
 np.str_('but'),
 np.str_('me'),
 np.str_('im'),
 np.str_('your'),
 np.str_('not'),
 np.str_('amp'),
 np.str_('out'),
 np.str_('its'),
 np.str_('will'),
 np.str_('an'),
 np.str_('no'),
 np.str_('has'),
 np.str_('fire'),
 np.str_('after'),
 np.str_('all'),
 np.str_('when'),
 np.str_('we'),
 np.str_('if'),
 np.str_('now'),
 np.str_('via'),
 np.str_('new'),
 np.str_('more'),
 np.str_('get'),
 np.str_('or'),
 np.str_('about'),
 np.str_('what'),
 np.str_('he'),
 np.str_('people'),
 np.str_('news'),
 np.str_('been'),
 n

In [46]:
# get the weight matrix of embedding layer
# these are the numerical representations of each token in our training data
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape

(10000, 128)

In [47]:
#create embedding fiels
import io

out_v = io.open("vectors.tsv", 'w', encoding="utf-8")
out_m = io.open("metadata.tsv", 'w', encoding="utf-8")

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue # skip 0, it's padding
  vec = embed_weights[index]
  out_v.write("\t".join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")

out_v.close()
out_m.close()

In [48]:
try:
  from google.colab import files
  files.download("vectors.tsv")
  files.download("metadata.tsv")
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [49]:
# RNN are useful for sequence data

# the premise of a rnn is
#to use the representation of a previous input to aid the representation of a later input

In [50]:
# LSTM - one of the most popular LSTM cells
# Input -> Tokenize -> Embedding -> Layers -> Output
# Create an Lstm model

from tensorflow.keras import layers
input = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.LSTM(units=64)(x)
print(x.shape)
#x = layers.LSTM(units=64, return_sequences=True)(x)
#print(x.shape)
#x = layers.LSTM(64, return_sequences=True)(x)
#print(x.shape)
#x = layers.LSTM(64, activation='relu')(x)
#print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")


(None, 15, 128)
(None, 64)


In [51]:
model_2.summary()

Model: "model_2_LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None)                 │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,329,473 (5.07 MB)

 Trainable params: 1,329,473 (5.07 MB)

 Non-trainable params: 0 (0.00 B)

In [52]:
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [53]:
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20250519-174229
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 15s 47ms/step - accuracy: 0.9064 - loss: 0.2874 - val_accuracy: 0.7769 - val_loss: 0.5474
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 16s 29ms/step - accuracy: 0.9418 - loss: 0.1532 - val_accuracy: 0.7835 - val_loss: 0.6311
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 12s 36ms/step - accuracy: 0.9535 - loss: 0.1203 - val_accuracy: 0.7769 - val_loss: 0.6552
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.9620 - loss: 0.1034 - val_accuracy: 0.7690 - val_loss: 0.8843
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.9689 - loss: 0.0828 - val_accuracy: 0.7795 - val_loss: 0.8119


In [54]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step


array([[3.6212035e-02],
       [9.1104740e-01],
       [9.9969786e-01],
       [7.8315921e-02],
       [8.4209762e-04],
       [9.9735558e-01],
       [9.1452390e-01],
       [9.9980259e-01],
       [9.9968106e-01],
       [3.0183968e-01]], dtype=float32)

In [55]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [56]:
model_2_results = caluclate_results(y_true=val_labels, y_pred=model_2_preds)
model_2_results

{'accuracy': 77.95275590551181,
 'precision': 0.7811151206076258,
 'recall': 0.7795275590551181,
 'f1': 0.7776506830100394}

In [57]:
# GRU (gated recurring unit)
# GRU is similar to LSTM but has less parameters

In [58]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(units=64)(x)
# x = layers.LSTM(42, return_sequences=True)(x)
# x = layers.GRU(99)(x)
#x = layers.Dense(64, activation="relu")(x)
#x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [59]:
model_3.summary()

Model: "model_3_GRU"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 64)             │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,317,313 (5.03 MB)

 Trainable params: 1,317,313 (5.03 MB)

 Non-trainable params: 0 (0.00 B)

In [60]:
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [61]:
model_3_history=model_3.fit(train_sentences,
                            train_labels,
                            epochs=5,
                            validation_data=(val_sentences, val_labels),
                            callbacks=[create_tensorboard_callback(SAVE_DIR, "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20250519-174330
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 12s 39ms/step - accuracy: 0.8877 - loss: 0.2756 - val_accuracy: 0.7730 - val_loss: 0.7424
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.9697 - loss: 0.0815 - val_accuracy: 0.7717 - val_loss: 0.8371
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 11s 44ms/step - accuracy: 0.9727 - loss: 0.0693 - val_accuracy: 0.7782 - val_loss: 1.1688
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 10s 44ms/step - accuracy: 0.9769 - loss: 0.0558 - val_accuracy: 0.7677 - val_loss: 1.2257
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step - accuracy: 0.9795 - loss: 0.0481 - val_accuracy: 0.7717 - val_loss: 1.3068


In [62]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step


array([[4.5332938e-04],
       [6.7954683e-01],
       [9.9989891e-01],
       [6.9860302e-02],
       [5.6597422e-05],
       [9.9987787e-01],
       [8.8420117e-01],
       [9.9996400e-01],
       [9.9992353e-01],
       [6.9626057e-01]], dtype=float32)

In [63]:
model_3_preds=tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [64]:
model_3_results = caluclate_results(y_true=val_labels, y_pred=model_3_preds)
model_3_results

{'accuracy': 77.16535433070865,
 'precision': 0.773559668677779,
 'recall': 0.7716535433070866,
 'f1': 0.7694523344399301}

In [65]:
# CNN

# we've used CNN for images but images are 2d and text is 1d
# typical structure of conv1d model is:
# Inputs (text) -> tokenization -> embedding
# -> layer(s) (typically conv1d + pooling)
# -> outputs (class probabiliteies)

In [66]:
# Conv1d model 5
from tensorflow.keras import layers

embedding_test = embedding(text_vectorizer(["this is a test sentence"]))

conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5,
                        activation="relu",
                        padding="valid")

conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output) # equivalent to get the most important feature
# or get the feature with the highest value"

max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.0480674 , 0.02733916, 0.02594895, 0.07482202, 0.02286592,
        0.05759742, 0.0249869 , 0.        , 0.08142636, 0.02338315,
        0.01976808, 0.05612836, 0.01386564, 0.03183301, 0.05793869,
        0.04503478, 0.02359063, 0.03837525, 0.07172237, 0.06953859,
        0.03179637, 0.03226019, 0.0265739 , 0.05530948, 0.        ,
        0.        , 0.04255922, 0.        , 0.0476745 , 0.05009651,
        0.04771484, 0.01997058]], dtype=float32)>

In [67]:
embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [68]:
from tensorflow.keras import layers

input = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(input)
x = embedding(x)
x = layers.Conv1D(filters=64,
                  kernel_size=5,
                  strides=1,
                  activation="relu",
                  padding="valid")(x)
x = layers.GlobalMaxPool1D()(x)
# outputs =  layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(input, outputs, name="model_5_conv1d")

model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_5.summary()

Model: "model_5_conv1d"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 11, 64)         │        41,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_1          │ (None, 64)             │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,321,089 (5.04 MB)

 Trainable params: 1,321,089 (5.04 MB)

 Non-trainable params: 0 (0.00 B)

In [69]:
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20250519-174427
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.9328 - loss: 0.1912 - val_accuracy: 0.7743 - val_loss: 0.8216
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.9701 - loss: 0.0817 - val_accuracy: 0.7677 - val_loss: 0.9575
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.9756 - loss: 0.0639 - val_accuracy: 0.7612 - val_loss: 1.0874
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.9793 - loss: 0.0526 - val_accuracy: 0.7677 - val_loss: 1.1456
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.9800 - loss: 0.0489 - val_accuracy: 0.7612 - val_loss: 1.1432


In [70]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


array([[1.2342912e-01],
       [9.3709022e-01],
       [9.9985743e-01],
       [4.2691696e-02],
       [5.5338302e-07],
       [9.9724048e-01],
       [9.8813069e-01],
       [9.9999094e-01],
       [9.9999917e-01],
       [7.5905240e-01]], dtype=float32)

In [71]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [72]:
model_5_results = caluclate_results(y_true=val_labels, y_pred=model_5_preds)
model_5_results

{'accuracy': 76.11548556430446,
 'precision': 0.7611448173771751,
 'recall': 0.7611548556430446,
 'f1': 0.7599510749117049}

In [80]:
!pip install tf_keras

In [82]:
import tensorflow_hub as hub


sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="Universal_sentence_encoder")


In [96]:
import tf_keras

model_6 = tf_keras.Sequential([
    sentence_encoder_layer,
    tf_keras.layers.Dense(64, activation="relu"),
    tf_keras.layers.Dense(1, activation="sigmoid")
], name="model_6_use")

In [97]:
model_6.compile(loss="binary_crossentropy",
                optimizer=tf_keras.optimizers.Adam(),
                metrics=["accuracy"])

In [98]:
model_6.summary()

Model: "model_6_use"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Universal_sentence_encoder  (None, 512)               256797824 
  (KerasLayer)                                                   
                                                                 
 dense_2 (Dense)             (None, 64)                32832     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [99]:
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 6s 19ms/step - loss: 0.5059 - accuracy: 0.7905 - val_loss: 0.4512 - val_accuracy: 0.8005
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.4140 - accuracy: 0.8139 - val_loss: 0.4412 - val_accuracy: 0.8018
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.4014 - accuracy: 0.8203 - val_loss: 0.4382 - val_accuracy: 0.8110
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3928 - accuracy: 0.8244 - val_loss: 0.4267 - val_accuracy: 0.8202
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3844 - accuracy: 0.8304 - val_loss: 0.4280 - val_accuracy: 0.8136


In [100]:
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 1s 16ms/step


array([[0.15816435],
       [0.80622977],
       [0.9899337 ],
       [0.20563103],
       [0.72924155],
       [0.6935567 ],
       [0.98357993],
       [0.9785111 ],
       [0.93864435],
       [0.10842066]], dtype=float32)

In [101]:
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [102]:
model_6_results = caluclate_results(y_true=val_labels, y_pred=model_6_preds)
model_6_results

{'accuracy': 81.36482939632546,
 'precision': 0.8163894248318297,
 'recall': 0.8136482939632546,
 'f1': 0.8119583027830023}

In [ ]:
##########################

In [103]:
len(train_df_shuffled)

7613

In [107]:
# transfer learning really helps when you don't have large enough dataset

train_10_percent = train_df_shuffled[['text', 'target']].sample(frac=0.1, random_state=42)
train_sentences_10_percent = train_10_percent['text'].to_list()
train_labels_10_percent = train_10_percent['target'].to_list()

len(train_sentences_10_percent), len(train_labels_10_percent)

(761, 761)

In [108]:
train_df_shuffled['target'].value_counts()

,count
target,
0,4342
1,3271


In [109]:
train_10_percent['target'].value_counts()

,count
target,
0,413
1,348


In [111]:
######################

model_7 = tf_keras.models.clone_model(model_6)


In [112]:
# model 7 is same model but it doesn't know anything

model_7.summary()

Model: "model_6_use"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Universal_sentence_encoder  (None, 512)               256797824 
  (KerasLayer)                                                   
                                                                 
 dense_2 (Dense)             (None, 64)                32832     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [113]:
model_7.compile(loss="binary_crossentropy",
                optimizer=tf_keras.optimizers.Adam(),
                metrics=["accuracy"])


In [114]:
model_7_history = model_7.fit(train_sentences_10_percent,
                              train_labels_10_percent,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
24/24 [==============================] - 5s 45ms/step - loss: 0.6667 - accuracy: 0.6978 - val_loss: 0.6252 - val_accuracy: 0.7966
Epoch 2/5
24/24 [==============================] - 1s 25ms/step - loss: 0.5907 - accuracy: 0.7963 - val_loss: 0.5435 - val_accuracy: 0.8031
Epoch 3/5
24/24 [==============================] - 1s 35ms/step - loss: 0.5144 - accuracy: 0.8003 - val_loss: 0.4773 - val_accuracy: 0.8202
Epoch 4/5
24/24 [==============================] - 1s 47ms/step - loss: 0.4604 - accuracy: 0.8121 - val_loss: 0.4341 - val_accuracy: 0.8202
Epoch 5/5
24/24 [==============================] - 1s 25ms/step - loss: 0.4261 - accuracy: 0.8200 - val_loss: 0.4053 - val_accuracy: 0.8346


In [115]:
model_7_pred_probs = model_7.predict(val_sentences)
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_results = caluclate_results(y_true=val_labels, y_pred=model_7_preds)
model_7_results

24/24 [==============================] - 1s 11ms/step


{'accuracy': 83.46456692913385,
 'precision': 0.8344984215925784,
 'recall': 0.8346456692913385,
 'f1': 0.834427063817841}